In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import pickle

metadata = pickle.load(open('metadata.pkl', "rb"))

In [4]:
import numpy as np
from math import ceil
import tensorflow as tf
from malaya_speech.train.model import fastspeech, conformervc
import malaya_speech






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [5]:
sbmt_i = metadata[0]
x_org = sbmt_i[2]
c = np.random.normal(size = (2, 512)).astype(np.float32)

In [6]:
mel = tf.placeholder(tf.float32, (None, None, 80))
ori_vector = tf.placeholder(tf.float32, (None, 512))
target_vector = tf.placeholder(tf.float32, (None, 512))
mel_lengths = tf.placeholder(tf.int32, (None))

In [7]:
dim_neck = 32
config = malaya_speech.config.fastspeech_config
config['encoder_hidden_size'] = 512 + 80
config['decoder_hidden_size'] = 512 + dim_neck
config = fastspeech.Config(vocab_size = 1, **config)
config_conformer = malaya_speech.config.conformer_small_encoder_config
config_conformer['subsampling']['type'] = 'none'

In [8]:
model = conformervc.model.Model(dim_neck, config_conformer, config, 80)

none
none


In [9]:
encoder_outputs, mel_before, mel_after, codes = model(mel, ori_vector, target_vector, mel_lengths)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
tf.trainable_variables()

[<tf.Variable 'conformervc/Encoder/conformer/conformer_linear/kernel:0' shape=(592, 144) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_linear/bias:0' shape=(144,) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_block_0/conformer_block_0_ff_module_1/conformer_block_0_ff_module_1_ln/gamma:0' shape=(144,) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_block_0/conformer_block_0_ff_module_1/conformer_block_0_ff_module_1_ln/beta:0' shape=(144,) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_block_0/conformer_block_0_ff_module_1/conformer_block_0_ff_module_1_dense_1/kernel:0' shape=(144, 576) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_block_0/conformer_block_0_ff_module_1/conformer_block_0_ff_module_1_dense_1/bias:0' shape=(576,) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_block_0/conformer_block_0_ff_module_1/conformer_block_0_ff_module_1_dens

In [11]:
codes_ = model.call_second(mel_after, ori_vector, mel_lengths)

In [12]:
encoder_outputs, mel_before, mel_after, codes, codes_

(<tf.Tensor 'conformervc/concat:0' shape=(?, ?, 544) dtype=float32>,
 <tf.Tensor 'conformervc/mel_before/BiasAdd:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'conformervc/add:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'conformervc/mul:0' shape=(?, ?, 32) dtype=float32>,
 <tf.Tensor 'mul:0' shape=(?, ?, 32) dtype=float32>)

In [13]:
loss_f = tf.losses.mean_squared_error
max_length = tf.cast(tf.reduce_max(mel_lengths), tf.int32)
mask = tf.sequence_mask(
    lengths = mel_lengths, maxlen = max_length, dtype = tf.float32
)
mask = tf.expand_dims(mask, axis = -1)
mask

<tf.Tensor 'ExpandDims_1:0' shape=<unknown> dtype=float32>

In [14]:
from functools import partial
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss

mse_mel = partial(
    loss_f,
    weights = mask
)

In [15]:
mel_loss_before = calculate_3d_loss(mel, mel_before, mse_mel)
mel_loss_before

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'mean_squared_error/value:0' shape=() dtype=float32>

In [16]:
mel_loss_after = calculate_3d_loss(mel, mel_after, mse_mel)
mel_loss_after

<tf.Tensor 'mean_squared_error_1/value:0' shape=() dtype=float32>

In [17]:
g_loss_cd = tf.losses.absolute_difference(codes, codes_)

In [18]:
loss = mel_loss_before + mel_loss_after + g_loss_cd

In [19]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [20]:
%%time

o = sess.run([encoder_outputs, mel_before, mel_after], feed_dict = {mel: [x_org] * 2,
                                                                    ori_vector: c,
                                                                    target_vector: c,
                                                                    mel_lengths: [len(x_org)] * 2})

CPU times: user 4.79 s, sys: 183 ms, total: 4.97 s
Wall time: 4.47 s


In [21]:
o[0].shape, o[1].shape, o[2].shape

((2, 90, 544), (2, 90, 80), (2, 90, 80))

In [22]:
o = sess.run([mel_loss_before, mel_loss_after, g_loss_cd], feed_dict = {mel: [x_org] * 2,
                                                                    ori_vector: c,
                                                                    target_vector: c,
                                                                    mel_lengths: [len(x_org)] * 2})

In [23]:
o

[1.4799279, 2.6570122, 1.1761347]

In [24]:
tf.trainable_variables()

[<tf.Variable 'conformervc/Encoder/conformer/conformer_linear/kernel:0' shape=(592, 144) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_linear/bias:0' shape=(144,) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_block_0/conformer_block_0_ff_module_1/conformer_block_0_ff_module_1_ln/gamma:0' shape=(144,) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_block_0/conformer_block_0_ff_module_1/conformer_block_0_ff_module_1_ln/beta:0' shape=(144,) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_block_0/conformer_block_0_ff_module_1/conformer_block_0_ff_module_1_dense_1/kernel:0' shape=(144, 576) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_block_0/conformer_block_0_ff_module_1/conformer_block_0_ff_module_1_dense_1/bias:0' shape=(576,) dtype=float32>,
 <tf.Variable 'conformervc/Encoder/conformer/conformer_block_0/conformer_block_0_ff_module_1/conformer_block_0_ff_module_1_dens

In [25]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, "test/model.ckpt")

'test/model.ckpt'

In [26]:
!ls -lh test

total 197000
-rw-r--r--  1 huseinzolkepli  staff    77B Feb  7 11:02 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    79M Feb  7 11:02 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    54K Feb  7 11:02 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff    17M Feb  7 11:02 model.ckpt.meta


In [27]:
!rm -rf test